In [299]:
import re
from gurobipy import *
import numpy as np

In [300]:
def read_file(filename):
    with open(filename, 'r') as file:
        content = file.read()  # Open the file in read mode

    data = {}  # Initialize data as an empty dictionary

    # Each file has number of inputs under which data is stored as a matrix
    numerical_data = re.findall(r'\[([\d\s.]+)\]', content)  # Extract values following each header as numerical values and remove '\t's and '\n's

    sections = ['Orders', 'Allocations', 'DistanceShelfShelf', 'DistancePackagingShelf', 'FullDistanceMatrix']

    for i, section in enumerate(sections):
        data[section] = []
        lines = numerical_data[i].strip().split('\n')
        for line in lines:
            values = line.strip().split()  # Split using spaces
            data[section].append([int(val) for val in values])  # Assuming all values are integers

    return data

In [301]:
def find_closest_product(current_position, products, distances):
    closest_product = None
    min_distance = float('inf')
    for product in products:
        if distances[current_position][product] <= min_distance:
            closest_product = product
            min_distance = distances[current_position][product]
    return closest_product

def visit_order(order, distances):
    visited = [0]
    current_position = 0  
    for k in range(len(order)):
            closest_product = find_closest_product(current_position, order, distances)
            visited.append(closest_product)
            order.remove(closest_product)
            current_position = closest_product
    
    visited.append(0)
    return visited

In [302]:
filename = r"Data_Xpress_FullDist.txt"
data = read_file(filename)

# Extracting data into arrays
Orders = data.get('Orders')
Allocations = np.asarray(data.get('Allocations'))
DistanceShelfShelf = np.asarray(data.get('DistanceShelfShelf'))
FullDistanceMatrix = np.asarray(data.get('FullDistanceMatrix'))

NbShelves = 96
Shelves = range(1, NbShelves + 1)

FullDistanceMatrix = np.roll(FullDistanceMatrix, shift = 1, axis = 1)
FullDistanceMatrix = np.roll(FullDistanceMatrix, shift = 1, axis = 0)

test_allocations = [45, 79, 39, 68, 73, 53, 19, 44, 16, 71, 27, 41,  2, 46, 60, 67, 56, 
                    83, 80, 57, 69, 55, 75, 34, 89, 12, 81, 62, 23, 26, 24, 86,  3, 17,
                    90, 58, 51, 25, 85, 65, 31, 11, 87, 10, 13, 70, 35, 32, 47,  6, 30,
                    21, 43, 64, 66, 78, 76, 61,  8, 72, 22, 18, 82, 14, 28,  4,  5, 84,
                    54, 48, 63, 29, 49, 74, 37, 36, 20, 38, 50,  7, 88,  9, 40, 77, 15,
                    1, 33, 59, 42, 52,  45, 79, 39, 68, 73, 53]


In [303]:
allocations = Allocations[0]

product_to_shelf = {}
for i, product_group in enumerate(allocations):
    if product_group != 0:  # Check if the element is not zero
        product_to_shelf[product_group] = i

# Convert order matrix from product indices to shelf indices
OrdersByShelf1 = []
for order in Orders:
    order_shelf_indices = []
    for product_index in order:
        if product_index in product_to_shelf:
            shelf_index = product_to_shelf[product_index]
            order_shelf_indices.append(shelf_index+1)
        else:
            order_shelf_indices.append(0)  # Product not found in allocation matrix
    OrdersByShelf1.append(order_shelf_indices)

In [304]:
TotalDistance1 = 0
routes1 = []
for i, order in enumerate(OrdersByShelf1):
    visited_order1 = visit_order(order, FullDistanceMatrix)
    routes1.append(visited_order1)
    # print(f"Order {i+1}:", visited_order)
    
    for i in range(len(visited_order1)-1):
        TotalDistance1 += FullDistanceMatrix[visited_order1[i]][visited_order1[i+1]]

In [305]:
allocations = test_allocations

product_to_shelf = {}
for i, product_group in enumerate(allocations):
    if product_group != 0:  # Check if the element is not zero
        product_to_shelf[product_group] = i

# Convert order matrix from product indices to shelf indices
OrdersByShelf2 = []
for order in Orders:
    order_shelf_indices = []
    for product_index in order:
        if product_index in product_to_shelf:
            shelf_index = product_to_shelf[product_index]
            order_shelf_indices.append(shelf_index+1)
        else:
            order_shelf_indices.append(0)  # Product not found in allocation matrix
    OrdersByShelf2.append(order_shelf_indices)

In [306]:
TotalDistance2 = 0
routes2 = []
for i, order in enumerate(OrdersByShelf2):
    visited_order2 = visit_order(order, FullDistanceMatrix)
    routes2.append(visited_order2)
    # print(f"Order {i+1}:", visited_order)

    for i in range(len(visited_order2)-1):
        TotalDistance2 += FullDistanceMatrix[visited_order2[i]][visited_order2[i+1]]

print(f"Total distance: {3*TotalDistance1}")
print(f"Total distance: {3*TotalDistance2}")

Total distance: 290706
Total distance: 293424


In [290]:
# allocation_matrix = [0] * 96  # Initialize with zeros
# product_to_shelf = {}

# for shelf_index, product in enumerate(test_allocations):
#     if product not in product_to_shelf:
#         product_to_shelf[product] = [shelf_index]  # Initialize with a list containing the current shelf index
#     else:
#         # If the product already exists in the dictionary, append the new shelf index to the list
#         product_to_shelf[product].append(shelf_index)

# # Convert product_to_shelf dictionary to a list of tuples if the product is assigned to multiple shelves
# product_to_shelf_tuples = {k: tuple(v) if len(v) > 1 else v[0] for k, v in product_to_shelf.items()}

# orders_matrix_shelf_indices = []
# for order in Orders:
#     order_shelf_indices = []
#     for product_index in order:
#         if product_index in product_to_shelf_tuples:
#             shelf_indices = product_to_shelf_tuples[product_index]
#             order_shelf_indices.append(shelf_indices)
#         else:
#             order_shelf_indices.append(0)  # Product not found in allocation matrix
#     orders_matrix_shelf_indices.append(order_shelf_indices)

In [280]:
# orders_matrix_shelf_indices

In [309]:
uncommon_routes = [i+1 for i in range(len(routes2)) if routes2[i]!=routes1[i]]
print(routes1[18], routes2[18])
# routes1 == routes2

[0, 25, 3, 37, 24, 59, 0] [0, 25, 37, 24, 93, 59, 0]


In [308]:
uncommon_routes

[7,
 12,
 19,
 27,
 35,
 46,
 49,
 60,
 61,
 70,
 81,
 93,
 95,
 101,
 102,
 125,
 148,
 153,
 173,
 174,
 182,
 186,
 221,
 228,
 235,
 237,
 243,
 250,
 253,
 283,
 293,
 302,
 316,
 326,
 340,
 343,
 352,
 355,
 359,
 376,
 377,
 379,
 380,
 388,
 401,
 426,
 427,
 434,
 438,
 439,
 459,
 461,
 463,
 482,
 483,
 495,
 516,
 533,
 555,
 581,
 582,
 584,
 585,
 593,
 600,
 605,
 606,
 607,
 610,
 618,
 644,
 645,
 654,
 664,
 666,
 667,
 676,
 682,
 685,
 686,
 709,
 728,
 745,
 749,
 750,
 753,
 757,
 758,
 770,
 773,
 785,
 800,
 807,
 826,
 836,
 844,
 845,
 847,
 856,
 857,
 864,
 875,
 879,
 885,
 886,
 890,
 897,
 904,
 933,
 935,
 979,
 980,
 987,
 1013,
 1018,
 1028,
 1038,
 1080,
 1095,
 1118,
 1119,
 1136,
 1139,
 1160,
 1164,
 1165,
 1184,
 1192,
 1213,
 1229,
 1251,
 1255,
 1263,
 1266,
 1275,
 1278,
 1279,
 1281,
 1288,
 1300,
 1318,
 1320,
 1335,
 1339,
 1345,
 1370,
 1385,
 1390,
 1392,
 1407,
 1444,
 1448,
 1449,
 1453,
 1482,
 1506,
 1512,
 1528,
 1534,
 1536,
 1547,
